#### 1. Déclaration des packages NuGet

In [1]:
// Déclaration des packages NuGet requis
#r "nuget: CsvHelper, 27.1.1"
#r "nuget: ClosedXML, 0.95.4"
#r "nuget: XPlot.Plotly.Interactive"
#r "nuget: Lucene.Net.Analysis.Common, 4.8.0-beta00013"


#### 2. Définition des paramètres

Il s'agit du coefficient et du nom du prof, d'éventuels paramètres de sécurité ou d'affichage.

In [2]:
// --- Paramètres globaux ---

// Nombre de projets
int numberOfProjects = 1;

// Nombre de champs d'évaluation par projet
// (par exemple 3 par projet si on a NoteCommunication / NoteThéorique / NoteTechnique...)
int[] nbEvalFieldsPerProject = { 3 }; 

// Pondération de la note du prof : si =1, la note du prof compte autant que l'ensemble des élèves réunis
decimal teacherWeight = 1m;

// Email du professeur (toi)
string professorEmail = "jsboige@gmail.com";


#### 3. Upload et Chargement des Inscriptions

In [3]:
using Microsoft.DotNet.Interactive;
using System.IO;
using CsvHelper;
using System.Globalization;
using CsvHelper.Configuration;

public class StudentRecord
{
    public string Prénom { get; set; }
    public string Nom { get; set; }
    public string Email { get; set; }
    
    // On sépare chaque “Sujet Projet” en deux propriétés
    public string SujetProjet1 { get; set; }
    public string SujetProjet2 { get; set; }
    
    public List<string> Sujets => new List<string> { SujetProjet1, SujetProjet2 };

    public List<decimal> Notes { get; set; } = new();
    public decimal Moyenne => Notes.Any() ? Notes.Average() : 0;
}




public class StudentMap : ClassMap<StudentRecord>
{
    public StudentMap()
    {
        Map(m => m.Prénom).Name("Prénom");
        Map(m => m.Nom).Name("Nom de famille");
        Map(m => m.Email).Name("Adresse de courriel");
        Map(m => m.SujetProjet1).Name("Groupes GenAI");
        // Map(m => m.SujetProjet2).Name("Sujet Projet 2");
    }
}


// Upload et lecture du fichier CSV des inscriptions
var studentFileInput = await Kernel.GetInputAsync(
    "Veuillez uploader le fichier CSV contenant les inscriptions des étudiants:",
    typeHint: "file");

display($"Fichier sélectionné : {studentFileInput}");

List<StudentRecord> studentRecords;
using (var studentReader = new StreamReader(studentFileInput))
{
    using (var studentCsv = new CsvReader(studentReader, CultureInfo.InvariantCulture))
    {
        studentCsv.Context.RegisterClassMap<StudentMap>();
        studentRecords = studentCsv.GetRecords<StudentRecord>().ToList();
    }
}



In [4]:

// Affichage des 5 premières lignes pour vérification
// display(studentRecords.Take(20), "application/json");
studentRecords.DisplayTable();
studentRecords.Skip(20).DisplayTable();

// Affichage des élèves en anomalie (non inscrits)

#### 4. Chargement des fichiers de notes

Dans une boucle, on charge les différents exports csv des fichiers produits par Google Forms.

In [15]:
using System.Collections.Generic;
using CsvHelper.Configuration;
using System.Globalization;
using System.IO;
using System.Linq;
using Microsoft.DotNet.Interactive.Formatting;


public class EvaluationRecord
{
    public DateTime Date { get; set; }
    public string Email { get; set; }
    public string Nom { get; set; }
    public string Prénom { get; set; }
    public string Groupe { get; set; }
    public string SujetLibre { get; set; }
    public int NoteCommunication { get; set; }
    public int NoteThéorique { get; set; }
    public int NoteTechnique { get; set; }
    public int NoteOrganisation { get; set; }
    public string PointsPositifs { get; set; }
    public string PointsNégatifs { get; set; }
    public string Recommandations { get; set; }

    public decimal Note => ((decimal) (NoteCommunication + NoteTechnique + NoteThéorique + NoteOrganisation) * 2)  / (decimal) NbEvalFields;

    public bool IsTeacher { get; set; } 
    public int NbEvalFields { get; set; } = 4;

}




public class EvaluationMap : ClassMap<EvaluationRecord>
{
    public EvaluationMap()
    {
        Map(m => m.Date).Name("Horodateur");
        Map(m => m.Email).Name("Adresse e-mail");
        Map(m => m.Nom).Name("Votre nom");
        Map(m => m.Prénom).Name("Votre prénom");
        Map(m => m.Groupe).Name("Groupe à évaluer");
        Map(m => m.SujetLibre).Name("Sujet de la présentation");
        Map(m => m.NoteCommunication).Name("Qualité de la présentation (communication, la forme)");
        Map(m => m.NoteThéorique).Name("Qualité théorique (principes utilisés, classe d'algorithmes, contexte et explications des performances et des problèmes, histoire etc.)");
        Map(m => m.NoteTechnique).Name("Qualité technique (livrables, commits, qualité du code, démos, résultats, perspectives)");
        Map(m => m.NoteOrganisation).Name("Organisation (planning, répartition des tâches, collaboration, intégration au projet Github) ");
        Map(m => m.PointsPositifs).Name("Points positifs de la présentation");
        Map(m => m.PointsNégatifs).Name("Points négatifs de la présentation");
        Map(m => m.Recommandations).Name("Recommandations pour s'améliorer");
    }
}
var fileInputs = new List<string>();
var evaluations = new List<List<EvaluationRecord>>();
for (int i = 0; i < numberOfProjects; i++)
{
    var fileInput = await Kernel.GetInputAsync($"Veuillez uploader le fichier CSV pour le projet {i + 1}:", typeHint: "file");
    using var evalReader = new StreamReader(fileInput);
    var csvConfig = new CsvConfiguration(CultureInfo.CurrentCulture)
    {
        Delimiter = ",",
        HasHeaderRecord = true,
        HeaderValidated = null,
        MissingFieldFound = null,
        BadDataFound = null,
        IgnoreBlankLines = true,
        TrimOptions = TrimOptions.Trim,
    };
    using var evalCsv = new CsvReader(evalReader, csvConfig);
    evalCsv.Context.RegisterClassMap<EvaluationMap>();
    var projectEvaluations = evalCsv.GetRecords<EvaluationRecord>().ToList();

    // Validation du sujet libre contre le groupe évalué (simplifié ici)
    foreach (var eval in projectEvaluations)
    {
        eval.NbEvalFields = nbEvalFieldsPerProject[i];
        if (eval.Email == professorEmail)
        {
            eval.IsTeacher = true;
        }
        
        var expectedSubject = studentRecords.FirstOrDefault(s => s.Sujets[i] == eval.Groupe)?.Sujets[i];
        if (expectedSubject != null && !eval.Groupe.Contains(expectedSubject))
        {
            display($"Attention: Le sujet '{eval.Groupe}' ne correspond pas au groupe '{eval.Groupe}' attendu pour '{expectedSubject}'.");
        }
    }

    // Afficher les données chargées pour vérification
    fileInputs.Add(fileInput);
    evaluations.Add(projectEvaluations);
}




In [16]:

for (int i = 0; i < numberOfProjects; i++)
{
   
    // Afficher les données chargées pour vérification
    display($"Evaluations du projet {System.IO.Path.GetFileName(fileInputs[i])}");
    //  evaluations[i].Take(5).ToList().ToDisplayString("text/html").DisplayAs("text/markdown");
    evaluations[i].DisplayTable();
    
}

#### 5. Définition des structures de croisement et validation des données

On commence par définir les structures de données qui vont permettre de rapprocher évaluations et inscriptions

In [18]:

using Lucene.Net.Analysis;
using Lucene.Net.Analysis.Core;
using Lucene.Net.Analysis.Standard;
using Lucene.Net.Analysis.Miscellaneous;
using Lucene.Net.Analysis.TokenAttributes;
using Lucene.Net.Util;
using System.IO;





public class GroupEvaluation
{
    public decimal TeacherWeight { get; set; }
    public string Groupe { get; set; }
    
    public decimal Moyenne
    {
        get
        {
            var studentEvals = Evaluations.Where(e => !e.IsTeacher).ToList();
            var teacherEvals = Evaluations.Where(e => e.IsTeacher).ToList();
            decimal studentAvg = studentEvals.Any() ? studentEvals.Average(e => e.Note) : 0;
            decimal teacherAvg = teacherEvals.Any() ? teacherEvals.Average(e => e.Note) : 0;
            decimal toReturn = studentEvals.Any() ? (studentAvg + (teacherEvals.Any() ? teacherAvg * TeacherWeight : 0)) / (1 + (teacherEvals.Any() ? TeacherWeight : 0)) : 0;
            return toReturn;
        }
    }

    
     public decimal EcartType
    {
        get
        {
            var studentEvals = Evaluations.Where(e => !e.IsTeacher).ToList();
            var teacherEvals = Evaluations.Where(e => e.IsTeacher).ToList();

            if (!studentEvals.Any()) return 0;

            decimal moyenne = Moyenne;
            decimal varianceEtudiant = studentEvals.Any() ? (decimal) studentEvals.Select(e => Math.Pow((double)(e.Note - moyenne), 2)).Average(): 0;
            decimal varianceProfesseur = teacherEvals.Any() ? (decimal) Math.Pow((double)(teacherEvals.Average(e => e.Note) - moyenne), 2) : 0;

            var combinedVariance = (double) ((varianceEtudiant + varianceProfesseur * TeacherWeight) / (1 + TeacherWeight));
            return (decimal)Math.Sqrt(combinedVariance);
        }
    }

        
    public decimal NoteRectifiée { get; set; }

    public List<EvaluationRecord> Evaluations { get; set; }
    public DateTime Date =>  Evaluations.Skip(Evaluations.Count / 2).First().Date;
    public List<StudentRecord> GroupMembers { get; set; }   
}

public class ProjectEvaluation
{

    public string ProfessorEmail {get; set;}

    public class CustomAnalyzer : Analyzer
    {
       private LuceneVersion version;
    
       public CustomAnalyzer(LuceneVersion version)
       {
           this.version = version;
       }
    
       protected override TokenStreamComponents CreateComponents(string fieldName, TextReader reader)
       {
           var tokenizer = new StandardTokenizer(version, reader);
           TokenStream tokenStream = new LowerCaseFilter(version, tokenizer);
           tokenStream = new ASCIIFoldingFilter(tokenStream);
           return new TokenStreamComponents(tokenizer, tokenStream);
       }
    }

    public List<GroupEvaluation> GroupedEvaluations { get; set; } = new List<GroupEvaluation>();

    public decimal Moyenne => GroupedEvaluations.Select(n => n.Moyenne * n.GroupMembers.Count).Sum() / GroupedEvaluations.Select(n => n.GroupMembers.Count).Sum();

    public decimal EcartType => (decimal) Math.Sqrt(GroupedEvaluations.Select(n => Math.Pow((double) (n.Moyenne - Moyenne), 2) * n.GroupMembers.Count).Sum() 
    / GroupedEvaluations.Select(n => n.GroupMembers.Count).Sum());

    public static string FoldAccents(string text)
    {
        using (var analyzer = new CustomAnalyzer(LuceneVersion.LUCENE_48))
        using (var reader = new StringReader(text))
        using (var tokenStream = analyzer.GetTokenStream("", reader))
        {
            var charTermAttribute = tokenStream.AddAttribute<ICharTermAttribute>();
            tokenStream.Reset();
            var output = string.Empty;
            while (tokenStream.IncrementToken())
            {
                output += charTermAttribute.ToString() + " ";
            }
            tokenStream.End();
            return output.Trim();
        }
    }


    public string ProjectName(string name)
    {
        var firstName = name.Split(" ").First().Split("-").First();
        firstName = firstName.Substring(0, Math.Min(5, firstName.Length));
        return FoldAccents(firstName.ToLowerInvariant().Trim());
    }

    public void FilterEvaluations()
    {
        // display($"Noms des inscrits : ");
        // foreach (var groupEval in GroupedEvaluations)
        // {
        //     foreach (var student in groupEval.GroupMembers)
        //     {
        //         display($"{student.Prénom} ({ProjectName(student.Prénom)}) {student.Nom} ({ProjectName(student.Nom)})");
        //     }
        // }
        foreach (var groupEval in GroupedEvaluations)
        {
            groupEval.Evaluations = groupEval.Evaluations.Where(e => this.IsValidEvaluation(groupEval, e)).ToList();
        }
    }


    public bool MatchNames(StudentRecord student, EvaluationRecord eval)
    {
        return (ProjectName(student.Nom) == ProjectName(eval.Nom) && ProjectName(student.Prénom) == ProjectName(eval.Prénom))
        || (ProjectName(student.Prénom) == ProjectName(eval.Nom) && ProjectName(student.Nom) == ProjectName(eval.Prénom));
    }

    public bool MatchNames(EvaluationRecord eval1, EvaluationRecord eval2)
    {
        return (ProjectName(eval1.Nom) == ProjectName(eval2.Nom) && ProjectName(eval1.Prénom) == ProjectName(eval2.Prénom))
        || (ProjectName(eval1.Prénom) == ProjectName(eval2.Nom) && ProjectName(eval1.Nom) == ProjectName(eval2.Prénom));
    }

    public bool IsValidEvaluation(GroupEvaluation groupEvaluation, EvaluationRecord eval)
    {
       // Vérifier que la note n'est pas attribuée avant la présentation
       // et que les notes ne sont pas extrêmement hautes ou basses sans justification
       if (eval.Note < 1 && eval.Note > 19) {
              display($"Evaluation écartée: {eval}, Note invalide ({eval.Note})");
            return false;
       }

       // Vérifier que la date est cohérente avec les autres évaluations du groupe
       if (groupEvaluation.Date >= eval.Date + TimeSpan.FromHours(5)){
              display($"Evaluation écartée: {eval}, Etudiant {eval.Prénom} ({this.ProjectName(eval.Prénom)}) {eval.Nom} ({this.ProjectName(eval.Nom)}) a évalué trop tôt ({eval.Date})");
              return false;
       } 
       if (groupEvaluation.Date < eval.Date - TimeSpan.FromHours(2)) {
                display($"Evaluation écartée: {eval}, Etudiant {eval.Prénom} ({this.ProjectName(eval.Prénom)}) {eval.Nom} ({this.ProjectName(eval.Nom)}) a évalué trop tard ({eval.Date})");
                return false;
        }

       // L'étudiant doit être inscrit.
       if (eval.Email != ProfessorEmail && !this.GroupedEvaluations.Exists(s => s.GroupMembers.Exists(s => MatchNames(s, eval)))) {
                           display($"Evaluation écartée: {eval}, Etudiant {eval.Prénom} ({this.ProjectName(eval.Prénom)}) {eval.Nom} ({this.ProjectName(eval.Nom)}) non inscrit ({eval.Email})");
                           
                           return false;
                       }


       // Interdiction de voter pour soi-même
       if(groupEvaluation.GroupMembers.Exists(s => MatchNames(s, eval))) {
                           display($"Evaluation écartée: {eval}, Etudiant {eval.Prénom} ({this.ProjectName(eval.Prénom)}) {eval.Nom} ({this.ProjectName(eval.Nom)}) membre du groupe évalué, ");
                           return false;
                       }

       // Interdiction aux doublons
       if (groupEvaluation.Evaluations.Count(e => MatchNames(e, eval)) > 1) {
                           display($"Evaluation écartée: {eval}, {eval.Prénom} ({this.ProjectName(eval.Prénom)}) {eval.Nom} ({this.ProjectName(eval.Nom)}) a évalué plusieurs fois le même groupe");
                           return false;
                       }

       return true;
    }

}



#### 6. Définition des structures de croisement et validation des données

Puis on effectue l'ensemble des étapes de calcul du workflow

In [19]:
using XPlot.Plotly;

// Apariement des élèves à partir de l'inscription

var projectEvaluations = new List<ProjectEvaluation>();  

for (int i = 0; i < evaluations.Count; i++)
{
    var projectEvaluation = new ProjectEvaluation(){ProfessorEmail = professorEmail};
    projectEvaluations.Add(projectEvaluation);

    
    var projetEval = evaluations[i];
    var groupProjectEvaluations = projetEval.GroupBy(e => e.Groupe).Select(g => new GroupEvaluation {
        Groupe = g.Key,
        Evaluations = g.ToList(),
        GroupMembers = studentRecords.Where(s => s.Sujets.Contains(g.Key)).ToList(),
        TeacherWeight = teacherWeight
    }).ToList();


    foreach (var gEval in groupProjectEvaluations)
    {
        // Vérifier que chaque groupe a une évaluation du professeur
        bool hasTeacherEvaluation = gEval.Evaluations.Any(e => e.IsTeacher);
        if (!hasTeacherEvaluation)
        {
            throw new Exception($"Le groupe '{gEval.Groupe}' n'a pas d'évaluation de la part du professeur. Veuillez compléter avant de continuer.");
        }
        //Vérifier que chaque groupe a au moins un membre
        bool hasMember = gEval.GroupMembers.Count>0;
        if (!hasMember)
        {
            throw new Exception($"Le groupe '{gEval.Groupe}' n'a pas de membre. Veuillez compléter avant de continuer.");
        
        }


    }
    
    projectEvaluation.GroupedEvaluations.AddRange(groupProjectEvaluations);
    
   
}




#### 7. Définition des paramètres de rectification

Définition pour chacun des projets de la moyenne et l'écart type à appliquer, et d'un barème d'ajustement fonction de la taille des groupes.

In [20]:


// Affichage des 5 premières lignes pour vérification
// display(studentRecords.Take(20), "application/json");
// studentRecords.DisplayTable();
// studentRecords.Skip(20).DisplayTable();



// for (int i = 0; i < numberOfProjects; i++)
// {
   
//     // Afficher les données chargées pour vérification
//     display($"Evaluations du projet {System.IO.Path.GetFileName(fileInputs[i])}");
//     //  evaluations[i].Take(5).ToList().ToDisplayString("text/html").DisplayAs("text/markdown");
//     evaluations[i].DisplayTable();
    
// }




for (int i = 0; i < evaluations.Count; i++)
{
    var projectEvaluation = projectEvaluations[i];

 // Affichage des élèves en anomalie, typiquement des élèves inscrits mais sans note.
    
    projectEvaluation.FilterEvaluations();


   
    // Affichage des groupes et des moyennes

    display($"Moyenne des notes pour le projet {i + 1}: {projectEvaluation.Moyenne}, écart type: {projectEvaluation.EcartType}");

    // Visualisation avec Plotly
    var chart = Chart.Plot(
        new Histogram
        {
            x = projectEvaluation.GroupedEvaluations.Select(n => n.Moyenne),
            autobinx = false,
            xbins = new Xbins { start = 0, end = 20, size = 0.5 }
        }
    );
    
    display(chart);

    // Affichage de la moyenne et l'écart type pour chacun des projets

}



In [27]:
//Déclaration des variables

// Paramétrage de bonus/malus en points par taille de groupe.
// Clé : taille du groupe, Valeur : nombre de points à ajouter (+) ou retrancher (-).
public static Dictionary<int, decimal> groupSizeAdjustments = new Dictionary<int, decimal>
{
    // [1] = +3.0m,  
    // [2] = +1.0m,  
    // [3] = 0.0m,   // valeur de référence
    // [4] = -1.0m,
    // [5] = -3.0m

    // [1] = +5.0m,  
    // [2] = +3.0m,  
    // [3] = +1.0m,   
    // [4] = 0.0m,
    // [5] = -1.0m,
    // [6] = -3.0m

    
    [1] = +1.0m,   
    [2] = 0.0m,
    [3] = -1.0m,
    [4] = -3.0m
};

// Méthode pour récupérer la valeur associée (ou une valeur par défaut si la clé n'existe pas).
public static decimal PalierGroupSizeAdjustment(int groupSize)
{
    // Si la taille du groupe n'est pas explicitement dans le dictionnaire, on applique un malus par défaut
    if (!groupSizeAdjustments.ContainsKey(groupSize))
    {
        throw new Exception($"Aucun ajustement de groupe n'est défini pour un groupe de taille {groupSize}.");
    }
    return groupSizeAdjustments[groupSize];
}



// Rectification en moyenne et en écart type

// Dans un premier temps, une valeur pour tous les projets
List<(decimal newMean, decimal newStdev)> rectificationParams = new List<(decimal newMean, decimal newStdev)>
{
    (12m, 1.5m),
    // (12.5m, 2m)
};



#### 8. Application des paramètres de rectification 

1. **Ajustement par la taille du groupe**  
   - Nous attribuons un **bonus** ou un **malus** en points, en fonction de la taille du groupe, via un tableau de correspondance.   
   - Par exemple, un groupe de 1 étudiant obtient `+1` point, un groupe de 4 obtient `–0.5`, etc.

2. **Rectification moyenne / écart-type**  
   - Nous recalculons la nouvelle moyenne générale du projet après l’ajustement de taille.  
   - Nous appliquons alors la méthode `AdjustGrade(...)` pour recadrer la note autour d’une moyenne cible (`rectification.newMean`) et d’un écart type cible (`rectification.newStdev`).

**Ordre de calcul**  
Cette approche assure que la note finale prenne bien en compte la taille du groupe **avant** d’effectuer la mise à l’échelle statistique. Ainsi, si un étudiant est seul, il bénéficie d’un léger bonus avant le centrage-réduction.


In [28]:
using System;
using ClosedXML.Excel;

// --- 1) Fonction de centrage-réduction déjà existante ---
public static decimal AdjustGrade(decimal originalGrade,
                                 decimal groupMean,
                                 decimal groupStdDev,
                                 decimal targetMean = 10,
                                 decimal targetStdDev = 2)
{
    if (groupStdDev == 0) // Pour éviter la division par zéro
        return targetMean; // Retourne la moyenne cible si l'écart-type est nul

    // Calcul de la note ajustée (centrage-réduction)
    decimal adjustedGrade = ((originalGrade - groupMean) / groupStdDev) * targetStdDev + targetMean;

    // Clamper la note pour qu'elle reste dans les bornes 0 à 20
    adjustedGrade = Math.Max(0, Math.Min(20, adjustedGrade));

    return adjustedGrade;
}

// --- 2) Application en deux étapes (bonus/malus, puis rectification) ---

// Étape A : Bonus/malus en fonction de la taille du groupe
foreach (var project in projectEvaluations)
{
    foreach (var groupEval in project.GroupedEvaluations)
    {
        // Moyenne brute (sans ajustement) telle que définie par groupEval.Moyenne
        decimal rawAvg = groupEval.Moyenne;

        // Calcul du palier (bonus/malus)
        int groupSize = groupEval.GroupMembers.Count;
        decimal bonusMalus = PalierGroupSizeAdjustment(groupSize);

        // Application du bonus/malus : on additionne
        decimal adjusted = rawAvg + bonusMalus;

        // On borne la note [0..20]
        adjusted = Math.Max(0, Math.Min(20, adjusted));

        // On stocke ce résultat temporaire dans NoteRectifiée
        groupEval.NoteRectifiée = adjusted;
    }
}

// Étape B : Rectification statistique (centrage-réduction)
//           On recalcule la moyenne et l'écart-type du projet
//           en se basant cette fois sur groupEval.NoteRectifiée.

for (int i = 0; i < projectEvaluations.Count; i++)
{
    var project = projectEvaluations[i];
    var (targetMean, targetStdev) = rectificationParams[i];

    // 1) Calcul d'une moyenne localement, à partir de NoteRectifiée de chaque groupe
    //    et en tenant compte du nombre de membres.
    int totalStudents = project.GroupedEvaluations.Sum(g => g.GroupMembers.Count);
    if (totalStudents == 0)
        continue;  // s'il n'y a pas d'étudiants, on évite la division par zéro

    double sumRectified = project.GroupedEvaluations
                                 .Sum(g => (double)g.NoteRectifiée * g.GroupMembers.Count);
    decimal projectMean = (decimal)(sumRectified / totalStudents);

    // 2) Calcul de l'écart-type local sur NoteRectifiée
    double sumSquaredDiffs = project.GroupedEvaluations
                                    .Sum(g => Math.Pow((double)(g.NoteRectifiée - projectMean), 2)
                                              * g.GroupMembers.Count);
    decimal projectStdev = (decimal)Math.Sqrt(sumSquaredDiffs / totalStudents);

    // 3) Application de la méthode AdjustGrade pour chaque groupe
    foreach (var groupEval in project.GroupedEvaluations)
    {
        decimal currentGrade = groupEval.NoteRectifiée; // note déjà ajustée par bonus/malus
        decimal finalGrade = AdjustGrade(currentGrade,
                                         projectMean,
                                         projectStdev,
                                         targetMean,
                                         targetStdev);

        groupEval.NoteRectifiée = finalGrade;
    }
}


### 9. Génération du spreadsheet de résultats

In [29]:
using ClosedXML.Excel;
using System;
using System.Linq;

public string WrapText(string text, int maxLength)
{
    var words = text.Split(' ');
    var wrappedText = new StringBuilder();
    string line = "";

    foreach (var word in words)
    {
        if ((line + word).Length > maxLength)
        {
            wrappedText.AppendLine(line.Trim());
            line = "";
        }
        line += word + " ";
    }

    if (line.Length > 0)
        wrappedText.AppendLine(line.Trim());

    return wrappedText.ToString();
}


public void GenerateWorkbook(List<ProjectEvaluation> projectEvaluations, List<StudentRecord> students)
{
    var workbook = new XLWorkbook();
    var summarySheet = workbook.AddWorksheet("Résumé Étudiants");

    // En-tête pour la feuille de résumé
    summarySheet.Cell(1, 1).Value = "Nom";
    summarySheet.Cell(1, 2).Value = "Prénom";
    int currentColumn = 3;
    for (int i = 0; i < projectEvaluations.Count; i++)
    {
        summarySheet.Cell(1, currentColumn).Value = $"Groupe Projet {i + 1}";
        summarySheet.Cell(1, currentColumn + 1).Value = $"Note Projet {i + 1}";
        currentColumn += 2;
    }
    summarySheet.Cell(1, currentColumn).Value = "Note Moyenne Finale";

    // Trier les étudiants par nom et prénom
    var sortedStudents = students.OrderBy(s => s.Nom).ThenBy(s => s.Prénom).ToList();
    int row = 2;
    foreach (var student in sortedStudents)
    {
        decimal totalNotes = 0;
        int numberOfGrades = 0;
        summarySheet.Cell(row, 1).Value = student.Nom;
        summarySheet.Cell(row, 2).Value = student.Prénom;

        currentColumn = 3;
        for (int i = 0; i < projectEvaluations.Count; i++)
        {
            var project = projectEvaluations[i];
            var eval = project.GroupedEvaluations.FirstOrDefault(g => g.GroupMembers.Contains(student));
            if (eval != null)
            {
                summarySheet.Cell(row, currentColumn).Value = eval.Groupe;
                summarySheet.Cell(row, currentColumn + 1).Value = eval.NoteRectifiée;
                summarySheet.Cell(row, currentColumn + 1).Style.NumberFormat.Format = "0.0";
                totalNotes += eval.NoteRectifiée;
                numberOfGrades++;
            }
            currentColumn += 2;
        }

        decimal finalAverage = numberOfGrades > 0 ? totalNotes / numberOfGrades : 0;
        summarySheet.Cell(row, currentColumn).Value = finalAverage;
        summarySheet.Cell(row, currentColumn).Style.NumberFormat.Format = "0.0";
        row++;
    }

     // Format de tableau
    var table = summarySheet.Range(1, 1, row - 1, currentColumn).CreateTable("SummaryTable");
    // summarySheet.Tables.Add(table);
    table.ShowTotalsRow = false;
    table.Theme = XLTableTheme.TableStyleMedium15;

    // Ajustement des colonnes
    summarySheet.Columns().AdjustToContents();



    // Feuilles de retour qualitatif pour chaque projet
    for (int i = 0; i < projectEvaluations.Count; i++)
    {
        var feedbackSheet = workbook.AddWorksheet($"Projet {i + 1} Feedback");
        feedbackSheet.Cell(1, 1).Value = "Groupe";
        feedbackSheet.Cell(1, 2).Value = "Points Positifs";
        feedbackSheet.Cell(1, 3).Value = "Points Négatifs";
        feedbackSheet.Cell(1, 4).Value = "Recommandations";

        var feedbacks = projectEvaluations[i].GroupedEvaluations;
        int feedbackRow = 2;
        foreach (var feedback in feedbacks)
        {
            foreach (var eval in feedback.Evaluations)
            {
                if (!string.IsNullOrWhiteSpace(eval.PointsPositifs) || !string.IsNullOrWhiteSpace(eval.PointsNégatifs) || !string.IsNullOrWhiteSpace(eval.Recommandations))
                {
                    feedbackSheet.Cell(feedbackRow, 1).Value = feedback.Groupe;
                    feedbackSheet.Cell(feedbackRow, 2).Value = WrapText(eval.PointsPositifs, 40);
                    feedbackSheet.Cell(feedbackRow, 3).Value = WrapText(eval.PointsNégatifs, 40);
                    feedbackSheet.Cell(feedbackRow, 4).Value = WrapText(eval.Recommandations, 40);
                    feedbackRow++;
                }
            }
        }

        // Format de tableau pour les feedbacks
        if (feedbackRow > 2) // S'il y a des données
        {
            var feedbackTable = feedbackSheet.Range(1, 1, feedbackRow - 1, 5).AsTable($"FeedbackTable{i + 1}");
            // feedbackSheet.Tables.Add(feedbackTable);
            feedbackTable.ShowTotalsRow = false;
            feedbackTable.Theme = XLTableTheme.TableStyleLight9;
        }

        // Ajustement automatique des colonnes
        feedbackSheet.Columns().AdjustToContents();

    }


    var savePath  = System.IO.Path.Combine(new System.IO.FileInfo(studentFileInput).DirectoryName, "Resultats_Evaluations.xlsx");
    workbook.SaveAs(savePath);
    display($"Fichier Excel généré : {savePath}");
}

GenerateWorkbook(projectEvaluations, studentRecords);
